In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader as ND2Reader
# import aicsimageio.vendor.omexml as ome
import glob
import time, sys
sys.path.insert(0,'/home/jmamede/scripts/notebooks/libraries/')
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide

In [4]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end
#For gui make a function with all this input for verification purposes when loading the files
dirname = dirname = "/home/jmamede/imscope/JM/2020/20200626/igfp1_caruby5_continue001trigger002.nd2"
# dirname = "D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2"

filelist = glob.glob(dirname)
filelist.sort()
# filelist = filelist[2:3]

print(filelist)

#Load the first file for reference and what's inside the file
frames = ND2Reader(filelist[0])
print(frames.sizes)
#Grab metadata
metadata = frames.metadata
print(metadata['channels'])

#If you don't have enough VRAM divide the XY to do the deconvolution that is automatically assembled after
xdivide = 2
ydivide = 2
depthdivide = depth_divide(xdivide, ydivide)


# print(frames.parser._raw_metadata.image_metadata[b'SLxExperiment'][b'ppNextLevelEx'][b''][b'ppNextLevelEx'][b''][b'uLoopPars'][b'dZStep'])


['/home/jmamede/imscope/JM/2020/20200626/igfp1_caruby5_continue001trigger002.nd2']
{'x': 2048, 'y': 2044, 'c': 3, 't': 67, 'z': 23, 'v': 31}
['FITC', 'mRuby3', 'miRFP670']


/home/jmamede/anaconda3/lib/python3.7/site-packages/nd2reader/common_raw_metadata.py:99: RuntimeWarning: Reported average frame interval (359362.9 ms) doesn't match the set interval (360000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/nd2reader/common_raw_metadata.py:99: RuntimeWarning: Reported average frame interval (900005.8 ms) doesn't match the set interval (900000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)
/home/jmamede/anaconda3/lib/python3.7/site-packages/nd2reader/common_raw_metadata.py:99: RuntimeWarning: Reported average frame interval (89999.6 ms) doesn't match the set interval (1800000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)


In [5]:
timesteps = frames.timesteps


In [13]:
i = 10
timesteps[i*23-23]/1000


101.71866654999927

In [3]:
#Need to make this a big more automatic, but indexing all the files and iterations is not easy
#In the version where the PSF is estimated by flowdec is easier because everything was set to automatic

PSFdir="/home/jmamede/Rush/PSF/"
psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
# psfgfp =  tf.imread(PSFdir+'/green.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647
remove = 8
remove = 15
psf647 = np.flip(psf647,
#                  axis=0
                )[remove:-remove]
# [8:-8]


psfgfp = np.flip(psfgfp,
#                  axis=0
                )[remove:-remove]
psfdapi = np.flip(psfdapi,
#                   axis=0
                 )[remove:-remove]
psfruby = np.flip(psfruby,
#                   axis=0
                 )[remove:-remove]

print(psf647.shape)
print(psfruby.shape)
# psf647

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


(5, 128, 128)
(5, 128, 128)


In [4]:
algo =init_RL_algo(psfgfp.ndim
#                    ,pad_mode='2357',pad_min=(1,1,1)
                   ,pad_mode='log2',pad_min=(1,1,1)
                  )


filelist

['/home/jmamede/Data/CaRuby3/VOG/Caru3_optineon.nd2',
 '/home/jmamede/Data/CaRuby3/VOG/Caruby_PSPAX.nd2',
 '/home/jmamede/Data/CaRuby3/VOG/iGFP_Caru3_3_3.nd2']

In [5]:
savePRJ = True
saveStack = False
maxT = None
for fname in filelist:
    frames =  ND2Reader(fname)
    frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

    channels =  frames.metadata['channels']
    metadata = frames.metadata
    
    
    xml = Nd2meta2OMEXML(frames, maxT=maxT)
    prjxml = Nd2meta2OMEXML(frames, project=True, maxT=maxT)

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)


    # frames.iter_axes = 'v'
    # for frame in frames:
    for visit in range(frames.sizes['v']):
#     for visit in range(21,31):

        frames.default_coords['v'] = visit
        stackfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'.ome.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'_PRJ.ome.tiff'
        if saveStack:
            tif = tf.TiffWriter(stackfile, bigtiff=True , imagej=False)
        if savePRJ:
            tifprj = tf.TiffWriter(prjfile, bigtiff=True, imagej=False)
        
        first = True
        for time in range(frames.sizes['t']):

            res = np.zeros(
                (frames.sizes['z'],frames.sizes['c']
                 ,frames.sizes['y'],frames.sizes['x'])
                , dtype=np.float32)
            for i in range(frames.sizes['c']):
                frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                ch = metadata['channels'][i]
                # ['a647', 'DAPI', 'GFP']
                # ['FITC', 'mRuby3', 'miRFP670']
                # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                # ['a647', 'DAPI', 'GFP']
                if ch == 'DAPI' or ch  == '470 nm':
                    psf = psfdapi
                elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM':
                    psf = psf647
                elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM':
                    psf = psfgfp
                elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM':
                    psf = psfruby
                elif ch == 'FRET-gYFP-dsRED':
                    psf = psfruby

                arr = da.from_array(frames[time]
#                                     [:,0:2044,0:2044]
#                                     [:,:,:]              
                                    , chunks=chunk_size)

#                     print(arr.shape)
                res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                    dtype='float32', algo=algo,
                                    psf=psf, iters=20
                                             ).compute(num_workers=1)
#                     res[:,i,:,:] = deconv(frames[time],
# #                                           depth  = depthdivide, boundary='reflect', dtype='float32',
#                                           algo=algo, psf=psf, iters=20
#                              ).compute(num_workers=1)

#                     res[i,:,:,:] = arr.map_blocks(
#                     deconv,dtype='float32').compute(num_workers=1)

                # If we need to swap Channel and Z in the future
                #             img5d = np.swapaxes(img5d,0,1)

                update_progress( (visit+1) * (time+1) / (frames.sizes['v']* frames.sizes['t']) )
                print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                        visit+1, frames.sizes['v']
                        ,time+1, frames.sizes['t']
                        , i+1, frames.sizes['c']))
# Make Function
            if saveStack:
                if first:
                    tif.save(res.astype(np.uint16),
#                         compress='LZMA',
                        description = xml.to_xml(),
                        photometric='minisblack',
                        #datetime= True,
                        metadata= None,
                        contiguous=False,
                        )
                    first = False
                else:
                    tif.save(res.astype(np.uint16),
#                     compress='LZMA',
#                     description = xml.to_xml(),
                    photometric='minisblack',
                    metadata= None,
                    contiguous=False,
                    )
            if savePRJ:
                if first:
                    tifprj.save(res.astype(np.uint16).max(axis=0),
#                         compress='LZMA',
                        description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:
                    tifprj.save(res.astype(np.uint16).max(axis=0),
#                         compress='LZMA',
#                         description = xml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )

        if saveStack: tif.close()
        if savePRJ: tifprj.close()

    update_progress(1)

Progress: [####################] 100.0%


In [6]:
# print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

<weakproxy at 0x7fe213c4ddd0 to Device at 0x7fe213e23b90>